In [1]:
import poke_battle_sim as pb
import copy


In [2]:
from poke_battle_sim.core.battle import Battle


class MinimaxNode:

    def __init__(self, state: pb.Battle) -> None:
        self.battle = state
        self.score = 0
        self.move1 = None
        self.move2 = None
        self.children = {}

    def copy(self):
        return copy.deepcopy(self)

    def set_name(self, name):
        self.action = name


In [3]:
from poke_battle_sim.core.pokemon import Pokemon


def selection_function(battle):
    battle.t1.current_poke = battle.t1.poke_list[battle.t1.poke_id]


class MinimaxTrainer(pb.Trainer):
    def __init__(self, name: str, poke_list: list[Pokemon], selection):
        super().__init__(name, poke_list, selection)
        self.poke_id = 0
        self.selection = selection_function

    def simulate_turn(self, battle: pb.Battle, move1, move2):     
        new_battle = copy.deepcopy(battle)
        new_battle.turn(
            t1_turn=self.get_translated_move_name(move1),
            t2_turn=self.get_translated_move_name(move2),
        )

        return new_battle

    def evaluate_state(self, battle: pb.Battle) -> int:
        s = 0
        if self.tailwind_count > 0:
            s += 100

        s += 5

        return s

    def get_opponent_stats(self, opponent):
        return opponent.copy()

    def get_translated_move_name(self, move):
        return ["move", move.name]

    def choose_move(self, battle: pb.Battle):
        new_battle = copy.deepcopy(battle)
        bestMove = None

        max_score = -float("Inf")

        for move in self.current_poke.moves:
            score = self.minimax(new_battle, move, 1, False)

            if score > max_score:
                max_score = score
                bestMove = move

        return self.get_translated_move_name(bestMove)

    def minimax(
        self, battle: pb.Battle, move, depth, isMaxPlayer
    ):

        if depth == 0 or battle.is_finished():
            score = self.evaluate_state(battle)

            return score

        if isMaxPlayer:
            max_score = -float("Inf")

            for cur_move in self.current_poke.moves + [0,1,2,3,4,5]:
                score = self.minimax(battle, cur_move, 1, False)

                max_score = max(score, max_score)

            return max_score

        else:
            min_score = float("Inf")

            for opp_move in battle.t2.current_poke.moves + [0, 1, 2, 3, 4, 5]:
                new_battle = self.simulate_turn(battle, move, opp_move)

                score = self.minimax(new_battle, None, depth - 1, True)

                min_score = min(score, min_score)

            return min_score

In [4]:
articuno = pb.Pokemon(
    "Articuno",
    10,
    ["tackle", "tailwind"],
    "male",
    stats_actual=[100, 50, 50, 50, 50, 55],
)

pikachu = pb.Pokemon(
    "Pikachu",
    13,
    ["thunder-shock", "growl"],
    "male",
    stats_actual=[100, 50, 50, 50, 50, 50],
)
pikachu2 = pb.Pokemon(
    "Pikachu",
    13,
    ["thunder-shock", "growl"],
    "male",
    stats_actual=[100, 50, 50, 50, 50, 50],
)
zapdos = pb.Pokemon(
    "Zapdos",
    13,
    ["thunder-shock", "growl"],
    "male",
    stats_actual=[100, 50, 50, 50, 50, 50],
)
ash = MinimaxTrainer("Ash", [articuno, pikachu2, zapdos], selection=None)


misty = pb.Trainer("Misty", [pikachu])
battle = pb.Battle(ash, misty)

In [5]:
battle.start()
while not battle.is_finished():
    ash.poke_id = 2
    battle.turn(t1_turn=["other", "switch"], t2_turn=["move", "thunder-shock"])

battle.get_all_text()

['Ash sent out ARTICUNO!',
 'Misty sent out PIKACHU!',
 'Turn 1:',
 'Ash sent out ZAPDOS!',
 'PIKACHU used Thunder Shock!',
 'Turn 2:',
 'Ash sent out ARTICUNO!',
 'PIKACHU used Thunder Shock!',
 "It's super effective!",
 "It's super effective!",
 'Turn 3:',
 'Ash sent out ZAPDOS!',
 'PIKACHU used Thunder Shock!',
 'Turn 4:',
 'Ash sent out ARTICUNO!',
 'PIKACHU used Thunder Shock!',
 "It's super effective!",
 "It's super effective!",
 'Turn 5:',
 'Ash sent out ZAPDOS!',
 'PIKACHU used Thunder Shock!',
 'A critical hit!',
 'Turn 6:',
 'Ash sent out ARTICUNO!',
 'PIKACHU used Thunder Shock!',
 "It's super effective!",
 'ARTICUNO fainted!',
 "PIKACHU's attack missed!",
 'Ash sent out ZAPDOS!',
 'Turn 7:',
 'Ash sent out PIKACHU!',
 'PIKACHU used Thunder Shock!',
 "It's not very effective...",
 "It's not very effective...",
 'Turn 8:',
 'Ash sent out ZAPDOS!',
 'PIKACHU used Thunder Shock!',
 'Turn 9:',
 'Ash sent out PIKACHU!',
 'PIKACHU used Thunder Shock!',
 "It's not very effective...

In [6]:
articuno.moves[0].power

40